# Linear Trees

In [2]:
pip install --upgrade linear-tree

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
from lineartree import LinearTreeRegressor
from sklearn.metrics import mean_absolute_error
import time

In [5]:
pip install PyArrow

Note: you may need to restart the kernel to use updated packages.


In [6]:
tiempo = pd.read_parquet('dataframe_final_1970-2023_alicante_alicante.parquet')

In [7]:
# 70% del tiempo para el train, 30% validacion (hasta 31 diciembre 2022)
# el conjunto de test cogeremos 2023

fecha_test = '2007-12-31'

In [8]:
# las variables del día siguiente las eliminamos y también las correspondientes a la localización ya que estamos solo en Alicante

model_columns = list(set(tiempo.columns) - set(['tmed_mañana', 'prec_mañana', 'tmin_mañana', 'horatmin_mañana', 'tmax_mañana',
                                                                                    'horatmax_mañana', 'dir_mañana', 'velmedia_mañana', 'racha_mañana', 'horaracha_mañana', 
                                                                                    'sol_mañana', 'presMax_mañana', 'horaPresMax_mañana', 'presMin_mañana', 'horaPresMin_mañana',
                                                                                    'indicativo', 'nombre', 'provincia', 'altitud', 'fecha']))

In [9]:
tiempo.horatmin = tiempo.horatmin.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax = tiempo.horatmax.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha = tiempo.horaracha.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaPresMax = tiempo.horaPresMax.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin = tiempo.horaPresMin.apply(lambda x: x.hour if x is not None else np.nan)

tiempo.horatmin_1 = tiempo.horatmin_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_2 = tiempo.horatmin_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_3 = tiempo.horatmin_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_mañana = tiempo.horatmin_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horatmax_1 = tiempo.horatmax_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_2 = tiempo.horatmax_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_3 = tiempo.horatmax_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_mañana = tiempo.horatmax_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horaracha_1 = tiempo.horaracha_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_2 = tiempo.horaracha_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_3 = tiempo.horaracha_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_mañana = tiempo.horaracha_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horaPresMax_1 = tiempo.horaPresMax_1.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_2 = tiempo.horaPresMax_2.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_3 = tiempo.horaPresMax_3.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_mañana = tiempo.horaPresMax_mañana.apply(lambda x: x.hour if x is not None else np.nan)

tiempo.horaPresMin_1 = tiempo.horaPresMin_1.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_2 = tiempo.horaPresMin_2.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_3 = tiempo.horaPresMin_3.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_mañana = tiempo.horaPresMin_mañana.apply(lambda x: x.hour if x is not None else np.nan)

In [10]:
tiempo['año'] = tiempo['fecha'].dt.year
tiempo['mes'] = tiempo['fecha'].dt.month
tiempo['dia'] = tiempo['fecha'].dt.day

In [11]:
train = tiempo[tiempo.fecha <= fecha_test].fillna(-1)
test = tiempo[tiempo.fecha > fecha_test].fillna(-1)

In [12]:
from sklearn.linear_model import LinearRegression

## Target: tmax_mañana

In [14]:
metrics = {}
for max_depth in [5, 10, 15, 20]:
    start_time = time.time()
    model = LinearTreeRegressor(max_depth = max_depth, base_estimator=LinearRegression())
    model.fit(train[model_columns],train.tmax_mañana);

    train_pred = model.predict(train[model_columns])
    test_pred = model.predict(test[model_columns])

    metrics['LT_'+ str(max_depth)] = {
        'Train_MAE': mean_absolute_error(train.tmax_mañana, train_pred),
        'Test_MAE': mean_absolute_error(test.tmax_mañana, test_pred),
        'Run_Time': time.time() - start_time
    }

metrics_LT = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_LT['delta'] = metrics_LT.Test_MAE - metrics_LT.Train_MAE
metrics_LT

,Run_Time,Train_MAE,Test_MAE,delta
LT_5,65.029189,1.463658,1.968609,0.504951
LT_10,67.254275,1.463658,1.968609,0.504951
LT_15,73.677303,1.463658,1.968609,0.504951
LT_20,72.653455,1.463658,1.968609,0.504951


In [15]:
metrics_LT.to_parquet('lt_tmax.parquet')

## Target: tmin_mañana

In [16]:
metrics = {}
for max_depth in [5, 10, 15, 20]:
    start_time = time.time()
    model = LinearTreeRegressor(max_depth = max_depth, base_estimator=LinearRegression())
    model.fit(train[model_columns],train.tmin_mañana);

    train_pred = model.predict(train[model_columns])
    test_pred = model.predict(test[model_columns])

    metrics['LT_'+ str(max_depth)] = {
        'Train_MAE': mean_absolute_error(train.tmin_mañana, train_pred),
        'Test_MAE': mean_absolute_error(test.tmin_mañana, test_pred),
        'Run_Time': time.time() - start_time
    }

metrics_LT = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_LT['delta'] = metrics_LT.Test_MAE - metrics_LT.Train_MAE
metrics_LT

,Run_Time,Train_MAE,Test_MAE,delta
LT_5,68.558478,1.215344,1.46806,0.252716
LT_10,71.985860,1.215344,1.46806,0.252716
LT_15,72.247759,1.215344,1.46806,0.252716
LT_20,70.365083,1.215344,1.46806,0.252716


In [17]:
metrics_LT.to_parquet('lt_tmin.parquet')

## Target: prec_mañana

In [18]:
metrics = {}
for max_depth in [5, 10, 15, 20]:
    start_time = time.time()
    model = LinearTreeRegressor(max_depth = max_depth, base_estimator=LinearRegression())
    model.fit(train[model_columns],train.prec_mañana);

    train_pred = model.predict(train[model_columns])
    test_pred = model.predict(test[model_columns])

    metrics['LT_'+ str(max_depth)] = {
        'Train_MAE': mean_absolute_error(train.prec_mañana, train_pred),
        'Test_MAE': mean_absolute_error(test.prec_mañana, test_pred),
        'Run_Time': time.time() - start_time
    }

metrics_LT = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_LT['delta'] = metrics_LT.Test_MAE - metrics_LT.Train_MAE
metrics_LT

,Run_Time,Train_MAE,Test_MAE,delta
LT_5,85.303407,1.500934,1.864288,0.363354
LT_10,87.478316,1.512284,1.897018,0.384734
LT_15,85.742507,1.512284,1.897018,0.384734
LT_20,86.678037,1.512284,1.897018,0.384734


In [19]:
metrics_LT.to_parquet('lt_prec.parquet')

## Target: sol_mañana

In [20]:
metrics = {}
for max_depth in [5, 10, 15, 20]:
    start_time = time.time()
    model = LinearTreeRegressor(max_depth = max_depth, base_estimator=LinearRegression())
    model.fit(train[model_columns],train.sol_mañana);

    train_pred = model.predict(train[model_columns])
    test_pred = model.predict(test[model_columns])

    metrics['LT_'+ str(max_depth)] = {
        'Train_MAE': mean_absolute_error(train.sol_mañana, train_pred),
        'Test_MAE': mean_absolute_error(test.sol_mañana, test_pred),
        'Run_Time': time.time() - start_time
    }

metrics_LT = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_LT['delta'] = metrics_LT.Test_MAE - metrics_LT.Train_MAE
metrics_LT

,Run_Time,Train_MAE,Test_MAE,delta
LT_5,63.946213,2.140632,2.508725,0.368093
LT_10,65.321449,2.140632,2.508725,0.368093
LT_15,61.032966,2.140632,2.508725,0.368093
LT_20,62.871936,2.140632,2.508725,0.368093


In [21]:
metrics_LT.to_parquet('lt_sol.parquet')